# Create Agent

### Initializing Project Client

In [1]:
import os, json
import pandas as pd
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet
from user_functions import user_functions

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str="eastus.api.azureml.ms;fac34303-435d-4486-8c3f-7094d82a0b60;rg-anksing-5269_ai;anksing-evaluate-project", # os.environ["PROJECT_CONNECTION_STRING"],
)

AGENT_NAME = "Seattle Tourist Assistant for bug bash"

# Adding Tools to be used by Agent 
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

### Create Agent

In [2]:
agent = project_client.agents.create_agent(
    model="gpt-4o-mini", # os.environ["MODEL_DEPLOYMENT_NAME"],
    name=AGENT_NAME,
    instructions="You are a helpful assistant",
    toolset=toolset,
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_nnIc7BqBlgykTKnRq9ecBr4s


### Create Thread

In [3]:
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_hBjsINqr3awmq7wOEiRbfFnc


## Conversation with Agent
Use below cells to have conversation with the agent
- `Create Message[1]`
- `Execute[2]`

### Create Message[1]

In [14]:
# Create message to thread

MESSAGE = "Can you send me an email with weather information for Seattle?"

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=MESSAGE,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_ROKhU9Ox0uzj3ZSPRrYEymFz


### Execute[2]

In [15]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

print(f"Run ID: {run.id}")

Sending email to recipient@example.com...
Subject: Weather Information for Seattle
Body:
I was unable to retrieve the current weather data for Seattle, WA. Please check a reliable weather website or app for the latest updates. Let me know if you need help with anything else!
Run finished with status: RunStatus.COMPLETED
Run ID: run_rsAlbV1ci6WrrJhT0uT4g5Gu


### List Messages

In [16]:
for message in project_client.agents.list_messages(thread.id, order="asc").data:
    print(f"Role: {message.role}")
    print(f"Content: {message.content[0].text.value}")
    print("-" * 40)

Role: MessageRole.USER
Content: Where is Space Needle?
----------------------------------------
Role: MessageRole.AGENT
Content: The Space Needle is located in Seattle, Washington, USA. It is an iconic observation tower and was built for the 1962 World's Fair. The Space Needle is situated in the Lower Queen Anne neighborhood and is a popular tourist attraction, offering panoramic views of the Seattle skyline and surrounding areas.
----------------------------------------
Role: MessageRole.USER
Content: How is the weather there ?
----------------------------------------
Role: MessageRole.AGENT
Content: I'm unable to access the current weather data for Seattle, WA at this time. However, you can check a weather website or app for the latest updates on the conditions there. If you need information about typical weather patterns for a specific time of year, I can help with that!
----------------------------------------
Role: MessageRole.USER
Content: Can you send me an email with weather in

# Evaluate

### Get data from agent

In [17]:
import json
from azure.ai.evaluation import AIAgentConverter

# Initialize the converter that will be backed by the project.
converter = AIAgentConverter(project_client)

thread_id = thread.id
run_id = run.id

converted_data = converter.convert(thread_id, run_id)
print(json.dumps(converted_data, indent=4))

{
    "query": [
        {
            "role": "system",
            "content": "You are a helpful assistant"
        },
        {
            "createdAt": "2025-03-18T07:02:42Z",
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Where is Space Needle?"
                }
            ]
        },
        {
            "createdAt": "2025-03-18T07:02:47Z",
            "run_id": "run_ondvmGyKhCWwkqBdY5NzhPkg",
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": "The Space Needle is located in Seattle, Washington, USA. It is an iconic observation tower and was built for the 1962 World's Fair. The Space Needle is situated in the Lower Queen Anne neighborhood and is a popular tourist attraction, offering panoramic views of the Seattle skyline and surrounding areas."
                }
            ]
        },
        {
   

In [ ]:
evaluation_data = []
run_ids = converter._list_run_ids_chronological(thread_id=thread.id)

for run_id in run_ids:
    run_data = converter.convert(thread_id, run_id)
    evaluation_data.append(run_data)

# write to file
file_path = os.path.join(os.getcwd(), "evaluation_data.jsonl")
pd.DataFrame(evaluation_data).to_json(
    file_path, orient="records", lines=True
)

### Setting up evaluator

In [19]:
from azure.ai.evaluation import ToolCallAccuracyEvaluator , AzureOpenAIModelConfiguration
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint="https://ai-anksingevaluatehub493534796867.openai.azure.com",
    api_key="9h4nJr7YARncVgGDBeI3KYTsKGUMnFTv965BIdAHG06HzuRgNA7qJQQJ99BAACYeBjFXJ3w3AAAAACOGELLv",
    api_version="2025-01-01-preview",
    azure_deployment="gpt-4o-mini",
)

tool_call_accuracy = ToolCallAccuracyEvaluator(model_config=model_config)

### Run Evaluator

In [20]:
from azure.ai.evaluation import evaluate

response = evaluate(
    data=file_path,
    evaluators={
        "tool_call_accuracy": tool_call_accuracy,
    },
    azure_ai_project={
        "subscription_id": "fac34303-435d-4486-8c3f-7094d82a0b60",
        "project_name": "anksing-evaluate-project",
        "resource_group_name": "rg-anksing-5269_ai",
    }
)
print("Evaluation Results:")
pprint(response)

[2025-03-18 00:23:38 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-03-18 00:23:38 -0700][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_nseclplk_20250318_002338_542260, log path: C:\Users\anksing\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_nseclplk_20250318_002338_542260\logs.txt
[2025-03-18 00:23:43 -0700][promptflow._sdk._orchestrator.run_submitter][WARNING] - 1 out of 3 runs failed in batch run.
 Please check out C:/Users/anksing/.promptflow/.runs/azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_nseclplk_20250318_002338_542260 for more details.


2025-03-18 00:23:38 -0700   78504 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-03-18 00:23:39 -0700   78504 execution.bulk     INFO     Finished 1 / 3 lines.
2025-03-18 00:23:39 -0700   78504 execution.bulk     INFO     Average execution time for completed lines: 0.76 seconds. Estimated time for incomplete lines: 1.52 seconds.
2025-03-18 00:23:42 -0700   78504 execution.bulk     INFO     Finished 2 / 3 lines.
2025-03-18 00:23:42 -0700   78504 execution.bulk     INFO     Average execution time for completed lines: 1.91 seconds. Estimated time for incomplete lines: 1.91 seconds.
2025-03-18 00:23:43 -0700   78504 execution.bulk     INFO     Finished 3 / 3 lines.
2025-03-18 00:23:43 -0700   78504 execution.bulk     INFO     Average execution time for completed lines: 1.55 seconds. Estimated time for incomplete lines: 0.0 seconds.
2025-03-18 00:23:43 -0700   78504 execution          ERROR    1/3 flow run failed, indexes

c:\Users\anksing\.conda\envs\agent-evals-bug-bash\Lib\site-packages\promptflow\_sdk\operations\_local_storage_operations.py:516: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '(Failed)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  outputs.fillna(value="(Failed)", inplace=True)  # replace nan with explicit prompt
C:\Users\anksing\Repos\azure\azure-sdk-for-python\sdk\evaluation\azure-ai-evaluation\azure\ai\evaluation\_evaluate\_batch_run\proxy_client.py:60: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result_df.replace("(Failed)", math.nan, inplace=True)


======= Combined Run Summary (Per Evaluator) =======

{
    "tool_call_accuracy": {
        "status": "Completed with Errors",
        "duration": "0:00:05.373063",
        "completed_lines": 2,
        "failed_lines": 1,
        "log_path": "C:\\Users\\anksing\\.promptflow\\.runs\\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_nseclplk_20250318_002338_542260"
    }
}


Evaluation Results:
{'metrics': {'tool_call_accuracy.tool_call_accuracy': 1.0},
 'rows': [{'inputs.query': [{'content': 'You are a helpful assistant',
                             'role': 'system'},
                            {'content': [{'text': 'Where is Space Needle?',
                                          'type': 'text'}],
                             'createdAt': '2025-03-18T07:02:42Z',
                             'role': 'user'},
                            {'content': [{'tool_call': {'function': {'arguments': {'location': 'Seattle, '
                                                     

In [ ]:
pprint(response)